# 1.  Imports and Data

In [1]:
from sklearn.decomposition import PCA
import matplotlib as mpl 
import numpy as np
import os
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler



from codes.utils import stratified_train_test_group_kfold
from codes.utils import pred2class
from codes.utils import model_test_regression
from codes.regression_codes import gbr_gridsearch



with open("Data/MoS2_Analysis_Data_trained2", "rb") as fp:   # Unpickling
    MoS2_Proj1_Class_Data = pickle.load(fp)

#features = MoS2_ImageNet_100_data[0]
df = pd.DataFrame(MoS2_Proj1_Class_Data)
print(df.keys())
T_target = np.array(list(df['T']))
features = np.array(list(df['ImageNet']))
sampleId = np.array(list(df['sampleId']))
train_val_sampleId, test_sampleId, train_val_Y, test_Y = train_test_split(sampleId, T_target, test_size=0.1, stratify = T_target, random_state=40)

0.10.1


# 2. Data Splitting and Scaling

In [2]:


X = features
Y = np.array(T_target)
groups = np.array(sampleId)




train_val_groups, train_val_X, train_val_Y, test_X, test_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=0)
train_val_Y = train_val_Y.flatten()
#val_Y = val_Y.flatten()
test_Y = test_Y.flatten()

print(train_val_X.shape)
print(train_val_Y.shape)
print(test_X.shape)
print(test_Y.shape)


oversample = RandomOverSampler(sampling_strategy='not majority')

scaler = StandardScaler().fit(train_val_X)
train_val_X = scaler.transform(train_val_X)
test_X = scaler.transform(test_X)

Index(['sampleId', 'sampleLabel', 'image', 'T', 'ImageNet', 'MicroNet'], dtype='object')


# 3. Runing Cross-Validation

In [6]:
N_estimators = [i for i in range(100, 700, 100)]

Learning_rate = [0.0, 0.001, 0.01, 0.1, 1]

Min_samples_split = [i for i in range(2, 15, 3)]

Max_depth =[6, 7, 8]# [2, 3, 4, 5, 6, 7]#8, 9, 10]


def cross_10_folds_gpr(train_val_X, train_val_Y):
    
    best_train = []
    best_val = []
    best_variables = []
    for fold in range(10):
        #train_X, val_X, train_Y, val_Y = train_test_split(train_val_X, train_val_Y, test_size=0.1, stratify = train_val_Y, random_state=fold+30)#39, 42
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(train_val_X, train_val_Y, train_val_groups, n_splits=10, test_fold=fold)
        
        model_path = f'regression/ImageNet/aug3/GBR_{fold}_model.sav'
        
        train_X, train_Y = oversample.fit_resample(train_X, train_Y)
        variables_best, performance_best = gbr_gridsearch(train_X, train_Y, val_X, val_Y, N_estimators, Learning_rate, Min_samples_split,Max_depth,fold, model_path)
        best_train.append(performance_best['accuracy_train'])
        best_val.append(performance_best['accuracy_val'])
        best_variables.append(variables_best)
        print(f'fold: {fold} done!')
    return best_train, best_val, best_variables


best_train, best_val, best_variables = cross_10_folds_gpr(train_val_X, train_val_Y)




print(best_train)
print(best_val)
print(f'train_acc_mean: {np.mean(best_train) :.3f}, std: {np.std(best_train) :.3f}, val_acc_mean: {np.mean(best_val):.3f}, std: {np.std(best_val):.3f}')

<class 'generator'>
best_train_acc: 0.3333333333333333, best_val_acc: 0.5
best_train_acc: 1.0, best_val_acc: 0.6666666666666666
best_train_acc: 0.9968253968253968, best_val_acc: 0.7083333333333334
best_train_acc: 1.0, best_val_acc: 0.75
fold: 0 done!
<class 'generator'>
best_train_acc: 0.3333333333333333, best_val_acc: 0.4583333333333333
best_train_acc: 0.9779874213836478, best_val_acc: 0.6666666666666666
best_train_acc: 1.0, best_val_acc: 0.7083333333333334
fold: 1 done!
<class 'generator'>
best_train_acc: 0.3333333333333333, best_val_acc: 0.4583333333333333
best_train_acc: 0.9905660377358491, best_val_acc: 0.5416666666666666
best_train_acc: 0.9937106918238994, best_val_acc: 0.5833333333333334
best_train_acc: 1.0, best_val_acc: 0.625
best_train_acc: 1.0, best_val_acc: 0.6666666666666666
best_train_acc: 1.0, best_val_acc: 0.7083333333333334
fold: 2 done!
<class 'generator'>
best_train_acc: 0.3333333333333333, best_val_acc: 0.5
best_train_acc: 0.9936507936507937, best_val_acc: 0.6666666

# 4. Model Testing

In [5]:
best_fold = 4

model_path = f'regression/ImageNet/aug3/GBR_{best_fold}_model.sav'

# Fitting different train in the same hyperparameters
best_test, root_mean_squared_error, confusion_matrix_test = model_test_regression(train_val_X, train_val_Y, train_val_groups,test_X, test_Y, best_fold, model_path)

print('best_test: ', best_test)
print('rmse: ', root_mean_squared_error)
print(f'test_acc_mean: {np.mean(best_test) :.3f}, std: {np.std(best_test) :.3f}, rmse_mean: {np.mean(root_mean_squared_error)}, rmse_std: {np.std(root_mean_squared_error)}')

print("......")

print('5th confusion matrix: ', [list(item) for item in confusion_matrix_test[4]])

<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
best_test:  [0.7407407407407407, 0.6296296296296297, 0.7037037037037037, 0.6296296296296297, 0.5925925925925926, 0.6296296296296297, 0.7037037037037037, 0.48148148148148145, 0.6666666666666666, 0.6296296296296297]
rmse:  [23.37475167130802, 26.564861582857386, 30.13081068946875, 30.5099771954552, 28.888600282476304, 26.933433020524586, 23.052812772196315, 30.430147084033756, 30.235067648625613, 28.349797177007]
test_acc_mean: 0.641, std: 0.068, rmse_mean: 27.847025912395292, rmse_std: 2.6736531326890915
......
5th confusion matrix:  [[0, 3, 0], [1, 10, 2], [0, 5, 6]]


In [6]:
cm_list = []
for cm in confusion_matrix_test:
    cm_list.append([list(item) for item in cm])
#print(cm_list)

print(len(cm_list))
print('avg cm: ', [list(item) for item in np.mean(cm_list, axis=0)])


10
avg cm:  [[0.4, 2.6, 0.0], [1.1, 9.7, 2.2], [0.1, 3.7, 7.2]]


In [3]:
best_fold = 4
PATH = f'Model/Reg/TrainedImageNet/Up/GBC_{best_fold}_model.sav'
#PATH = os.path.join('Model', model_path)
loaded_model = pickle.load(open(PATH, 'rb'))
loaded_model.get_params()
#loaded_model.set_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'squared_error',
 'max_depth': 6,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': 1,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}